In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation, NMF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [2]:
csv_file_path = '/home/lucas/UFOP/ple_2020/analise_midias_sociais/final-work/data/all_pp_posts_bolsonaro_posts.csv'
df_candidate = pd.read_csv(csv_file_path,encoding = 'utf-8')

In [3]:
df_candidate.head(15)

,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
0,2020-03-22T18:47:55+0000,211857482296579_1826486634166981,agor videoconferenc president jair bolsonar pr...,- agora por videoconferencia o presidente jair...,23151,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,129465,147,318,21,112,10345,True
1,2020-03-22T16:30:39+0000,211857482296579_1826339070848404,inscrico seleca profission atuaca program medi...,- as inscricoes para a selecao de 5.800 profis...,5703,mobile_status_update,None,52238,128,61,5,43,2514,True
2,2020-03-22T13:37:26+0000,211857482296579_1826130007535977,ministeri saud ministr mandett reunia virtual ...,- estamos no ministerio da saude com o ministr...,11818,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p180x54...,97986,73,141,16,65,6292,True
3,2020-03-22T11:07:01+0000,211857482296579_1825937284221916,govern federal distribu milho test rap covid t...,- o governo federal distribui milhoes de teste...,39284,mobile_status_update,None,165539,88,393,20,68,16747,True
4,2020-03-22T02:06:48+0000,211857482296579_1825422034273441,cham precauca medic barat cas venh ser comprov...,- isso se chama precaucao . o medicamento e ba...,27236,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9052732...,162513,968,318,88,509,20723,True
5,2020-03-21T22:22:48+0000,211857482296579_1825061764309468,reconhec seriedad moment temor muit brasileir ...,- reconheco a seriedade do momento e o temor d...,40794,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/9065012...,305358,918,478,266,1267,48834,True
6,2020-03-21T18:36:01+0000,211857482296579_1824728371009474,hospital albert einstein possivel cur pacient ...,- hospital albert einstein e a possivel cura d...,201803,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/75...,215237,480,815,60,431,34278,True
7,2020-03-21T11:33:30+0000,211857482296579_1824261207722857,dois ded pros ratinh assist link youtub,- dois dedos de prosa com ratinho . - assista ...,33971,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,106117,336,317,113,624,15368,True
8,2020-03-21T02:37:00+0000,211857482296579_1823872341095077,assin med provisor deix clar ser competenc fed...,- assinei medida provisoria deixando claro ser...,9174,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,112873,181,200,106,1121,7582,True
9,2020-03-21T00:16:41+0000,211857482296579_1823743497774628,via eduard bolsonar,via eduardo bolsonaro,8150,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-0/p600x60...,79762,435,111,13,160,6480,True


In [4]:
n_total_posts = len(df_candidate)
print('número total de posts: {0}'.format(str(n_total_posts)))

número total de posts: 3646


In [7]:
n_posts_with_textual_messages = df_candidate[df_candidate.has_textual_message == True].shape[0]
n_posts_without_textual_messages = df_candidate[df_candidate.has_textual_message == False].shape[0]

print('número de posts com mensagens textuais: {0}'.format(str(n_posts_with_textual_messages)))
print('número de posts sem mensagens textuais: {0}'.format(str(n_posts_without_textual_messages)))

número de posts com mensagens textuais: 3394
número de posts sem mensagens textuais: 252


In [8]:
# Devemos remover esses posts sem menssagem antes de continuar a nossa análise?
# renomear depois o df caso sim

#filter_mask  = df_candidate['has_textual_message'] == True
#df_candidate = df_candidate[filter_mask]
#print('número de posts do df desconsiderando os posts sem menssagens textuais: {0}'.format(str(len(df_candidate))))

In [9]:
# separando o df de treinamento (30%) e o df de testes(70%) de maneira 'embaralhada'
df_training, df_test = train_test_split(df_candidate,train_size=0.7,test_size=0.3,shuffle=True)

In [10]:
print('número de posts do conjunto de treinamento (70%): {0}'.format(str(len(df_training))))
df_training.head(5)

número de posts do conjunto de treinamento (70%): 2552


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
3510,2014-02-13T17:39:16+0000,211857482296579_1467455580133314,hoj noit h horari d brasil bat pap viv olav ca...,hoje a noite as 22h ( horario d brasilia ) bat...,157,shared_story,https://external.xx.fbcdn.net/safe_image.php?d...,748,0,0,0,0,0,True
237,2020-02-20T00:02:26+0000,211857482296579_1785580654924246,aprov mp agro finaliz cam aprovaca mp agro imp...,- aprovada mp do agro . - finalizamos na camar...,5886,added_photos,https://scontent.xx.fbcdn.net/v/t1.0-9/s720x72...,49404,33,113,1,27,3405,True
2523,2016-07-25T21:59:59+0000,211857482296579_663456960469960,hoj centr rio janeir fascist golpist nao passa...,hoje no centro do rio de janeiro . `` fascista...,18331,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/13...,59167,4282,108,27,91,1997,True
485,2020-01-03T11:32:54+0000,211857482296579_2727400643949282,,,2696,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/73...,33758,250,97,24,63,5211,False
2941,2015-10-28T12:26:36+0000,211857482296579_550767191738938,felip mour brasil entrev flavi bolsonar psol q...,felipe moura brasil entrevista flavio bolsonar...,1289,shared_story,https://scontent.xx.fbcdn.net/v/t45.1600-4/121...,6963,0,0,0,0,0,True


In [11]:
print('número de posts do conjunto de teste (30%): {0}'.format(str(len(df_test))))
df_test.head(5)

número de posts do conjunto de teste (30%): 1094


,created_time,id,pre_processed_message,message_min_processed,shares,status_type,full_picture,reactions_like,reactions_haha,reactions_wow,reactions_sad,reactions_angry,reactions_love,has_textual_message
2054,2017-05-27T21:45:38+0000,211857482296579_846185195530468,tv bandeir londrin corrig apen nad contr minor...,- tv bandeirantes em londrina . - a corrigir a...,3063,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/18...,15587,244,38,2,8,1009,True
577,2019-11-14T22:00:23+0000,211857482296579_547702162660125,,,25607,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/72...,112903,783,381,115,333,20335,False
1353,2018-06-22T15:09:26+0000,211857482296579_1096224167193235,parabens polic milit estad rio janeir,- parabens policia militar do estado do rio de...,3959,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/35...,9231,21,173,2,1,572,True
1560,2018-03-07T21:38:58+0000,211857482296579_1025801967568789,,,12558,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/p5...,32872,296,205,25,130,5466,False
55,2020-03-15T20:55:59+0000,211857482296579_1815804728568505,manaus am,- manaus / am :,10291,added_video,https://scontent.xx.fbcdn.net/v/t15.5256-10/83...,44276,97,144,11,66,5466,True


In [12]:
#============================================================================================
# realizando o tf idf no df['pre_processed_message'] de treinamento
#============================================================================================

# instanciando o objeto para fazer o tf-idf
tfidf_vectorizer = TfidfVectorizer(analyzer='word')

# converte os documentos em matrizes
tfidf_matrix = tfidf_vectorizer.fit_transform(df_training['pre_processed_message'])

# obtendo as palavras (tokens)
tfidf_tokens = tfidf_vectorizer.get_feature_names()

# gerando o tf-idf
# colunas são as palavras processadas
# linhas são os documentos (post 1, post 2,post 3,post 4)
df_tfidfvect = pd.DataFrame(data = tfidf_matrix.toarray(),columns = tfidf_tokens)


print("\nTF-IDF\n")
df_tfidfvect.head(10)


TF-IDF



,abaix,abandon,abastec,abat,abav,abdelmassih,abenco,aberraca,abert,abertur,...,zec,zeland,zer,zl,zoe,zoeir,zon,zueir,zuer,zumb
0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.162055,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
# Aplicando uma máscara para identificar o tf-idf de certas palavras

filter_mask  = df_tfidfvect['zoeir'] != 0.0
df_tfidf_token_mask = df_tfidfvect[filter_mask]

print(df_tfidf_token_mask)

      abaix  abandon  abastec  abat  abav  abdelmassih  abenco  aberraca  \
1093    0.0      0.0      0.0   0.0   0.0          0.0     0.0       0.0   
1801    0.0      0.0      0.0   0.0   0.0          0.0     0.0       0.0   

      abert  abertur  ...  zec  zeland  zer   zl  zoe     zoeir  zon  zueir  \
1093    0.0      0.0  ...  0.0     0.0  0.0  0.0  0.0  0.656828  0.0    0.0   
1801    0.0      0.0  ...  0.0     0.0  0.0  0.0  0.0  0.477162  0.0    0.0   

      zuer  zumb  
1093   0.0   0.0  
1801   0.0   0.0  

[2 rows x 6513 columns]


In [17]:
#============================================================================================
# Implementacao do lda
#============================================================================================

In [21]:
cv = CountVectorizer(max_df = 0.8, min_df = 2)
df = cv.fit_transform(df_training['pre_processed_message'])

In [22]:
model_lda = LatentDirichletAllocation(n_components = 10)
model_lda.fit(df)

LatentDirichletAllocation()

In [23]:
for index, topic in enumerate(model_lda.components_):
    print(f'Top 10 words for #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Top 10 words for #0
['hoj', 'escol', 'rio', 'ser', 'brasileir', 'pt', 'educaca', 'public', 'brasil', 'milit']


Top 10 words for #1
['sao', 'nao', 'agradec', 'brasil', 'apoi', 'obrig', 'consideraca', 'jair', 'tod', 'bolsonar']


Top 10 words for #2
['milit', 'polit', 'canal', 'vid', 'youtub', 'tod', 'pov', 'brasileir', 'nao', 'brasil']


Top 10 words for #3
['anos', 'ministr', 'bolsonar', 'direit', 'estad', 'president', 'econom', 'brasil', 'ministeri', 'govern']


Top 10 words for #4
['liv', 'president', 'seman', 'sobr', 'vot', 'jair', 'nao', 'youtub', 'link', 'bolsonar']


Top 10 words for #5
['eduard', 'acim', 'carl', 'jair', 'abrac', 'nao', 'obrig', 'tod', 'brasil', 'bolsonar']


Top 10 words for #6
['tod', 'cam', 'hoj', 'brasil', 'lei', 'eduard', 'nao', 'bolsonar', 'deput', 'federal']


Top 10 words for #7
['pod', 'regional', 'avanc', 'desenvolv', 'ministeri', 'bilho', 'nao', 'ministr', 'govern', 'brasil']


Top 10 words for #8
['milit', 'verdad', 'pt', 'paul', 'flavi', 'deput', 'n